<a href="https://colab.research.google.com/github/vrandachag/SSY_24/blob/main/debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install texlive-fonts-recommended

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-fonts-recommended is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
!sudo apt install texlive-latex-extra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-latex-extra is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
!sudo pip3 install pylatex
!sudo pip3 install pdflatex==0.1.3
!sudo apt-get install latexmk
!sudo apt-get install -y texlive-latex-extra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
latexmk is already the newest version (1:4.76-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-latex-extra is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [4]:
!pip install PyLaTeX

In [5]:
!pip install pdflatex

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
import xlrd
import os
import pandas as pd
import numpy as np
import urllib.request
import cv2
from google.oauth2 import service_account
from googleapiclient.discovery import build
from pylatex import Document, Section, Figure, NoEscape, Command
from pylatex.utils import NoEscape
import gspread as gs
import requests
import re
from PIL import Image
from io import BytesIO
import json
import google.auth.transport.requests
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# from pylatex import Document, Section, Subsection, Figure, NoEscape, Command
# from pylatex.utils import bold, italic

In [8]:
df = pd.read_excel("/content/gdrive/MyDrive/SSY_SEVA_24/IDY -2024 - small.xlsx")

In [9]:
# df.set_index(['Country','State'], inplace=True)

In [10]:
filtered_df = df[df['Country'] == 'India']

In [11]:
sorted_df = filtered_df.sort_values(by=['State'])

In [12]:
sorted_df.shape

(3, 36)

In [13]:
sorted_df.columns

Index(['Unnamed: 0', 'Timestamp', 'Country', 'State', 'Organiser's Name(s)',
       'Organiser's Email ', 'Organiser's Mobile Number',
       'Organiser's Teacher/Volunteer Code',
       'Venue/Institute Name (Name of the place where event was organised)\n',
       'Type of Institute', 'Name of the Institute Contact Person',
       'Telephone Number of the Institute Contact Person',
       'City/Town Organised in\n', 'No. of Participants\n',
       'Date of the event', 'Upload Max 2 Pictures of the Event',
       'Upload a video of the Event', 'Chief Guest Name ',
       'Name of the Country', 'Organiser's Name(s).', 'Organiser's Email.',
       'Organiser's Mobile Number.', 'Organiser's Teacher/Volunteer Code.',
       'Venue/Institute Name (Name of the place where event was organised).\n',
       'Type of Institute.', 'Name of the Institute Contact Person.',
       'Telephone Number of the Institute Contact Person.',
       'City/Town Organised in.\n', 'Number of Participants',
     

In [14]:
def convert_drive_link(shared_link):
    # Check if the link contains 'open?id=' format
    if 'open?id=' in shared_link:
        file_id = shared_link.split('open?id=')[-1]
    elif '/d/' in shared_link:
        # Handle if the link is already in the direct access format
        file_id = shared_link.split('/d/')[-1].split('/')[0]
    else:
        raise ValueError("The provided link is not a valid Google Drive link.")

    # Construct the direct access link
    direct_access_link = f"https://drive.google.com/file/d/{file_id}/view"
    return direct_access_link

In [15]:
# Define the scope
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

# Path to your service account key file
SERVICE_ACCOUNT_FILE = '/content/gdrive/MyDrive/SSY_SEVA_24/ssy-seva-24.json'

# Extract the file ID from the provided Google Drive link
def extract_file_id(drive_link):
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', drive_link)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid Google Drive link")

# Fetch the file name using Google Drive API
def get_file_name(file_id):
    # Authenticate with the service account
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    # Build the Google Drive service
    service = build('drive', 'v3', credentials=credentials)

    # Get the file metadata
    file = service.files().get(fileId=file_id, fields='name').execute()

    return file.get('name')

In [16]:
# Create a new LaTeX document
doc = Document(documentclass='article')

In [17]:
for state, group in sorted_df.groupby('State'):
    #print(state,group)
    with doc.create(Section(state)):
        #to add column with number of participants
        total_participants = group['No. of Participants\n'].sum()
        # print("doc create")
        # total_participants = 1000 #temp
        num_venues = group['State'].nunique()
        doc.append(NoEscape(f"Outreach: {total_participants}\n"))
        doc.append(NoEscape(f"\nVenue: {num_venues}\n"))

        for index, row in group.iterrows():
          image_path = row['Upload Max 2 Pictures of the Event'].split(',')
          print("image_path",image_path)
          print("Hello why are you not working")
          for images in image_path:
            print(images)
            drive_link = convert_drive_link(images)
            try:
              file_id = extract_file_id(drive_link)
              print("file id:",file_id)
              file_name = get_file_name(file_id)
              print("file name:",file_name)
              print(f'The file name is: {file_name}')
            except Exception as e:
              print(f'An error occurred: {str(e)}')
              continue #change
            # Check if the file is an image
            # if file_name.endswith('.jpg') or file_name.endswith('.jpeg') or file_name.endswith('.png'):
            file_path = r'/content/gdrive/MyDrive/SSY_SEVA_24/National Pics/'+file_name
            # Add the image
            with doc.create(Figure(position='h!')) as fig:
              print("Hello image writing")
              fig.add_image(file_path, width=NoEscape(r'0.8\textwidth'))
              # fig.add_image(file_path, width='100px')
              venue_name = 'Venue/Institute Name (Name of the place where event was organised)\n'
              city_name = 'City/Town Organised in\n'
              fig.add_caption(f"{row[venue_name]},{row[city_name]}")
              # fig.append(NoEscape(r'\centering'))
              fig.append(Command('vspace','0.3cm'))

image_path ['https://drive.google.com/open?id=1V2ncuLUasw13JkJZyEZqggbAKbQX4drH', ' https://drive.google.com/open?id=1saiMCIBTXSrcQd8LJRbB8HjEvhwVOZzW', ' https://drive.google.com/open?id=1FCuhLH0u_eZm6r-UnmGozKC3-ZHRjAFK']
Hello why are you not working
https://drive.google.com/open?id=1V2ncuLUasw13JkJZyEZqggbAKbQX4drH
file id: 1V2ncuLUasw13JkJZyEZqggbAKbQX4drH
file name: IMG-20240607-WA0027 - Rushikesh Aoley.jpg
The file name is: IMG-20240607-WA0027 - Rushikesh Aoley.jpg
Hello image writing
 https://drive.google.com/open?id=1saiMCIBTXSrcQd8LJRbB8HjEvhwVOZzW
file id: 1saiMCIBTXSrcQd8LJRbB8HjEvhwVOZzW
file name: IMG-20240607-WA0024 - Rushikesh Aoley.jpg
The file name is: IMG-20240607-WA0024 - Rushikesh Aoley.jpg
Hello image writing
 https://drive.google.com/open?id=1FCuhLH0u_eZm6r-UnmGozKC3-ZHRjAFK
file id: 1FCuhLH0u_eZm6r-UnmGozKC3-ZHRjAFK
file name: IMG-20240607-WA0025 - Rushikesh Aoley.jpg
The file name is: IMG-20240607-WA0025 - Rushikesh Aoley.jpg
Hello image writing
image_path ['ht

In [18]:
doc.generate_tex()

In [19]:
doc.generate_pdf(r'/content/gdrive/MyDrive/SSY_SEVA_24/idk_debug_00', clean_tex=False,compiler='pdflatex')

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(/content/gdrive/MyDrive/SSY_SEVA_24/idk_debug_00.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/fontenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texmf/tex/latex/lm/lmodern.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/textcomp.sty)
(/usr/share/texlive/texmf-dist/tex/latex/lastpage/lastpage.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphics.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/trig.sty

CalledProcessError: Command '['pdflatex', '--interaction=nonstopmode', '/content/gdrive/MyDrive/SSY_SEVA_24/idk_debug_00.tex']' returned non-zero exit status 1.

In [ ]:
!pdflatex --interaction=nonstopmode /content/gdrive/MyDrive/SSY_SEVA_24/idk_01.tex